In [ ]:
import tensorflow as tf
import string
import requests


In [ ]:
response=requests.get("http://www.gutenberg.org/cache/epub/5200/pg5200.txt")

In [ ]:
response.text[:1500]


In [ ]:
data = response.text.split('\n')
data[0]


In [ ]:
data = data[253:]
data[0]


In [ ]:
len(data)

In [ ]:
data = " ".join(data)
data[:1000]


In [ ]:
def clean_text(doc):
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens
tokens = clean_text(data)
print(tokens[:50])

In [ ]:
len(tokens)

In [ ]:
length = 50 + 1
lines = []
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > 200000:
        break
print(len(lines))


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)


In [ ]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]


In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
y = to_categorical(y, num_classes=vocab_size)


In [ ]:
seq_length = X.shape[1]
seq_length

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X, y, batch_size = 256, epochs = 100)

In [ ]:

seed_text=lines[12343]
seed_text


In [ ]:
  def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')
    
     y_predict = model.predict_classes(encoded)
    
      predicted_word = ''
        for word,index in tokenizer.word_index.items():
          if index == y_predict:
        predicted_word = word
        break
    
seed_text = seed_text + ' ' + predicted_word
text.append(predicted_word)
return ' '.join(text)


In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)